# Set up

In [1]:
import os
import pandas as pd
pd.set_option("display.max_colwidth", None)

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dalle3-swo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e51119f8d8774069a6594d92ccf7a70d"

In [2]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Optional
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0
) 

gpt_35_turbo_16k = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
) 

smart_llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4",
    temperature=0
) 
gpt_35_turbo_instruct = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-instruct",
    temperature=0
) 
# Embedding
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-15-preview",
)

# Text loader

Use Azure AI document intelligence to extract text from word file and then refine output (such as add missing markdown syntax to header. Azure ai DI sometimes can not detect the heading level 2,3) and put into a markdown file and copy content to text file --> read text file, so that it can keep markdown syntax

This current implementation of a loader using Document Intelligence can incorporate content page-wise and turn it into LangChain documents. The default output format is markdown, which can be easily chained with MarkdownHeaderTextSplitter for semantic document chunking. You can also use mode="single" or mode="page" to return pure texts in a single page or document split by page.

Drawback:
- can not extract the list syntax, link (just get name,not link)
- can not extract text in image (chart, flow exported as image)

In [4]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = [
    "../../data/qna_update_data/PSG Programme Handbook.docx"
             ]
endpoint = "https://di-buv.cognitiveservices.azure.com/"
key = "fe9f945227944764889fd95f3bb2abda"

documents = []
for path in file_path:
    loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=path, api_model="prebuilt-layout", mode="markdown"
    )
    documents = documents + loader.load()

In [5]:
# Use word file
print(documents[0].page_content)

PERSONAL & SOCIAL GROWTH (PSG) PROGRAMME

HANDBOOK

This handbook provides BUV students with key information about the Personal and Social Growth (PSG) Programme, including the programme introduction, benefits for students' growth, main offerings with respective PSG points and rewards, frequently asked questions (FAQs), and other information.

Students are encouraged to read this handbook thoroughly, consult PSG Team and other student support teams to select suitable PSG activities, and design an individualised development plan at the beginning of every academic year. To get the latest information about PSG activities with points and rewards, as well as to review and adjust the PSG plan based on different development paths, distinct characteristics and aspirations, students should regularly revisit the handbook throughout their years at BUV.

TABLE OF CONTENTS

I. INTRODUCTION 3

1\. Personal and Social Growth (PSG) programme Overview 3

2\. Benefits of the PSG Programme 3

3\. Partici

Read text file

In [6]:
from langchain_community.document_loaders.text import TextLoader
loader = TextLoader("../../data/qna_update_data/psg_handbook_23_24.txt")
refined_documents = loader.load()

In [ ]:
print(refined_documents[0].page_content)

# Splitter

In [8]:
# Parent splitter - Mardown splitter 
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
handbook_docs = markdown_splitter.split_text(refined_documents[0].page_content)

In [9]:
len(handbook_docs)

31

In [10]:
handbook_df = pd.DataFrame({
    "page_content": [doc.page_content for doc in handbook_docs],
    "file_name": 31*['PSG Programme Handbook.pdf'],
    "page": 31*[0]
})

In [11]:
handbook_df.to_excel('psg_programme_handbook_23_24.xlsx', index=False)